In [40]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
# Retina display
%config InlineBackend.figure_format = 'retina'
from tqdm import tqdm

In [2]:
from tueplots import bundles
plt.rcParams.update(bundles.beamer_moml())

# Also add despine to the bundle using rcParams
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False

# Increase font size to match Beamer template
plt.rcParams['font.size'] = 12
# Make background transparent
plt.rcParams['figure.facecolor'] = 'none'

In [10]:
import os
# Set enviornment variable TORCH_HOME to specify where to download pretrained models
os.environ['TORCH_HOME'] = os.path.expanduser('~/TORCH_HOME')

In [46]:
import torchvision.transforms as transforms
import torch

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [32]:
# Download CIFAIR10 dataset in ~/TORCH_HOME/datasets/CIFAR10
from torchvision.datasets import CIFAR10

ds_cifar = CIFAR10(root=os.path.expanduser('~/TORCH_HOME/datasets'), 
                   transform=transform,
                   download=True)

Files already downloaded and verified


In [33]:
ds_cifar

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: /home/nipun.batra/TORCH_HOME/datasets
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [34]:
# Split dataset into train, pool and test
# train: 10 images per class
# test: 100 images per class
# pool: remaining images

from torch.utils.data import Subset
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Get indices of all images in the dataset
indices = np.arange(len(ds_cifar))

# Shuffle indices
np.random.shuffle(indices)

# Split indices into train, pool and test

# 10 images per class in train
train_indices = []
for i in range(10):
    train_indices.extend(indices[np.array(ds_cifar.targets) == i][:10])

# 100 images per class in test
test_indices = []
for i in range(10):
    test_indices.extend(indices[np.array(ds_cifar.targets) == i][10:110])

# Remaining images in pool
pool_indices = np.setdiff1d(indices, np.concatenate([train_indices, test_indices]))


In [35]:
indices.shape

(50000,)

In [36]:
indices[np.array(ds_cifar.targets) == 0][:10]

array([33109, 36480, 21123, 17523,  2530, 27578, 46915,  9801,  9405,
        7905])

In [37]:
# Print number of images in each set
print(f'Number of images in train: {len(train_indices)}')
print(f'Number of images in test: {len(test_indices)}')
print(f'Number of images in pool: {len(pool_indices)}')

Number of images in train: 100
Number of images in test: 1000
Number of images in pool: 48900


In [38]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [43]:
# image size
ds_cifar[0][0].shape

torch.Size([3, 32, 32])

In [47]:
# Simple CNN model 
# https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [50]:
net(torch.randn(1, 3, 32, 32)).shape

torch.Size([1, 10])

In [51]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)


In [52]:
for epoch in range(5):
    running_loss = 0.0
    for i, data in enumerate(tqdm(trainloader, desc=f'Epoch {epoch+1}'), 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()

NameError: name 'trainloader' is not defined